In [1]:
from datetime import date, timedelta
import os
import dill as pickle
import sys
import pandas as pd
import numpy as np
sys.path.append("..")

from abautomator.visualizer import BasicVisualizer, AbsDiffVisualizer, RelDiffVisualizer, utils
from abautomator import config, describer, analyzer

from bokeh.io import output_notebook
from bokeh.plotting import show, save
# from bokeh.models import MultiChoice, CustomJS, Button
from bokeh.layouts import column, row

output_notebook()
pd.set_option('display.max_colwidth', None)

Loading BokehJS ...

In [2]:
EXP_NAME = "ProtedtUpSellExp20224023_"
CTRL_NAME = 'ctrl'
COND_ORDER = ["tx_w_protect_upsell"]

In [3]:
def filter_coll_metrics(coll, column, value):
    for metric in coll.metrics:
        metric.user_metric_df = metric.user_metric_df[metric.user_metric_df[column] == value]
    
    return coll

In [4]:
local_coll = pickle.load(
    open(os.path.join("..", "tests", "cache", f"{EXP_NAME}_coll.p"), "rb" )
)

In [5]:
import copy

for value in ["all", "android", "ios"]:
    curr_coll = copy.deepcopy(local_coll)
    if value != "all":
        curr_coll = filter_coll_metrics(curr_coll, column="device_type", value=value)

    print(f"describing data for {value}")
    desc = describer.Describer(
        metrics=curr_coll.metrics
    )
    outcomes_dict = desc.describe_data(exp_name=EXP_NAME)
    
    print(f"analyzing data for {value}")
    analy =  analyzer.Analyzer(
        outcomes=outcomes_dict,
        ctrl_name=CTRL_NAME,
    )

    pickle.dump(
        analy, open(f"{EXP_NAME}_analy_{value}.p", "wb" )
    )

describing data for all
analyzing data for all
describing data for android
analyzing data for android
describing data for ios
analyzing data for ios


In [7]:
PRIMARY_METRIC_LIST = ['n_feed_views',
 'pct_feed_views',
 'n_feed_taps',
 'pct_feed_taps',
 'n_all_feed_shares',
 'pct_all_feed_shares',
 'n_direct_feed_shares',
 'pct_direct_feed_shares',
 'n_indirect_feed_shares',
 'pct_indirect_feed_shares',
 'n_signup_activation',
 'pct_signup_activation',
]
 
SECONDARY_METRIC_LIST = [
 'n_all_sessions',
 'pct_all_sessions',
 'n_organic_sessions',
 'pct_organic_sessions',
 'n_push_driven_sessions',
 'pct_push_driven_sessions',
 'n_incident_views',
 'pct_incident_views',
 'n_incident_share_attempts',
 'pct_incident_share_attempts',
]

GUARDRAIL_METRIC_LIST = [
 'n_chats',
 'pct_chats',
 'n_friend_invites',
 'pct_friend_invites',
 'n_trial_starts',
 'pct_trial_starts',
]

In [8]:
raw_df = analy.get_rel_diff_confidence_intervals()
output_df = raw_df.sort_values(by=['abs_mean'], ascending=False)#[["metric", "abs_mean"]]

In [9]:
df = utils.order_categories(output_df, primary_metric_list, COND_ORDER)
df = utils.remove_categories_w_no_order(df)
source = utils.convert_df_to_source(df)

vis = RelDiffVisualizer(source, title="Primary Metrics - Android")
fig = vis.get_figure()
to_show = vis.get_layout()
show(to_show)

NaN values found!


In [34]:
save(to_show, "/home/jovyan/work/notebooks/android-primary-metrics.html")

/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/home/jovyan/work/notebooks/android-primary-metrics.html'

In [35]:
df = utils.order_categories(output_df, secondary_metric_list, cond_order)
df = utils.remove_categories_w_no_order(df)
source = utils.convert_df_to_source(df)

vis = RelDiffVisualizer(source)
to_show = vis.get_layout()
show(to_show)

In [36]:
save(to_show, "/home/jovyan/work/notebooks/android-secondary-metrics.html")

/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/home/jovyan/work/notebooks/android-secondary-metrics.html'

In [37]:
df = utils.order_categories(output_df, guardrail_metric_list, cond_order)
df = utils.remove_categories_w_no_order(df)
source = utils.convert_df_to_source(df)

vis = RelDiffVisualizer(source)
to_show = vis.get_layout()
show(to_show)

In [38]:
save(to_show, "/home/jovyan/work/notebooks/android-guardrail-metrics.html")

/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/conda/lib/python3.9/site-packages/bokeh/io/saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/home/jovyan/work/notebooks/android-guardrail-metrics.html'